In [1]:
import requests
from bs4 import BeautifulSoup
import re
import pandas as pd

In [2]:
url = 'https://www.cars.com/shopping/results/?dealer_id=&keyword=&list_price_max=&list_price_min=&makes[]=&maximum_distance=150&mileage_max=100000&page_size=100&sort=best_match_desc&stock_type=all&year_max=&year_min=&zip=55423'
page = requests.get(url)
page

<Response [200]>

In [3]:
soup = BeautifulSoup(page.text, 'lxml')

In [4]:
columns = ['Year','Make','Model','Used_or_New','Price','ConsumerRating','ConsumerReviews','SellerType','SellerName','SellerRating','SellerReviews',
           'StreetName','State','Zipcode','DealType','ComfortRating','InteriorDesignRating','PerformanceRating','ValueForMoneyRating',
           'ExteriorStylingRating','ReliabilityRating','ExteriorColor','InteriorColor','Drivetrain','MinMPG','MaxMPG','FuelType',
           'Transmission','Engine','VIN','Stock','Mileage']   

df= pd.DataFrame(columns=columns)

In [5]:
for i in range(12):
    
    postings = soup.find_all('a', class_ = 'vehicle-card-link js-gallery-click-link')
    current_page = soup
    for post in postings:
        try:
            link = 'http://cars.com' + post.get('href')
            info_page = requests.get(link)
            soup = BeautifulSoup(info_page.text, 'lxml')
            
            ###DELETE THIS WHEN YOURE DONE
            #test_link = 'https://www.cars.com/vehicledetail/1d26a033-1886-4e9c-80cb-fb4f15ff4d51/'
            #info_page = requests.get(test_link)
            #soup = BeautifulSoup(info_page.text, 'lxml')
            
            
            #Data we want
            title_info = soup.find('h1', class_ = 'listing-title').text.split()
            year = title_info[0]
            make = title_info[1]
            model = ' '.join(title_info[2:])
            used_or_new = soup.find('p', class_ = 'new-used').text
            price = soup.find('span', class_ = 'primary-price').text
            consumer_rating = float(soup.find('section', class_= 'sds-page-section vehicle-reviews').find('span', class_='sds-rating__count').text)
            consumer_reviews = int(soup.find('section',class_= 'sds-page-section vehicle-reviews').find('a',class_= 'sds-rating__link sds-button-link').text[1:].split()[0])
            
            #Seller Details
            #Dealer
            try:
                seller_name = soup.find('h3', class_ = 'sds-heading--5 heading seller-name').text
                seller_rating = float(soup.find('section',class_ = 'sds-page-section seller-info').find('span', class_ = 'sds-rating__count').text)
                seller_reviews = soup.find('a', class_ = 'sds-rating__link sds-button-link').text.split()[0][1:]
                seller_type = 'Dealer'
                if ',' in seller_reviews:
                    seller_reviews = int(''.join(seller_reviews.split(',')))
                else:
                    seller_reviews =  int(seller_reviews)
                address_pieces = soup.find('div', class_ = 'dealer-address').text.split(',')
                street_name = address_pieces[0]
                state = address_pieces[1].split()[0]
                zipcode = address_pieces[1].split()[1] 
                
            #Private seller    
            except:
                seller_type = 'Private'
                pieces = soup.find('div', class_ = 'seller-address').text.split('from')
                seller_name = pieces[0].strip()
                address_pieces = pieces[1].split(',')
                street_name = address_pieces[0].strip()
                state = address_pieces[1].split()[0]
                zipcode = address_pieces[1].split()[1] 
    
            
            #Deal type
            deal_type = 'NA'
            first_badge = soup.find('span', class_ = 'sds-badge__label').text
            if 'deal' in first_badge.casefold():
                deal_type = first_badge.split()[0]
            
            #Rating breakdown
            rb = soup.find_all('div', class_ = 'review-breakdown')[0].find_all('span', class_ = 'sds-definition-list__value')
            
            comfort = float(rb[0].text)
            interior_design = float(rb[1].text)
            performance = float(rb[2].text) 
            value_for_money = float(rb[3].text)
            exterior_styling = float(rb[4].text)
            reliability = float(rb[5].text)
            
            #Basic Description Table
            bd = soup.find_all('div', class_ = 'basics-content-wrapper')[0].find_all('dd')
            
            exterior_color = bd[0].text.strip()
            interior_color = bd[1].text.strip()
            drive_train = bd[2].text.strip()
            min_mpg = int(bd[3].text.split('\n')[2].split('–')[0])
            max_mpg = int(bd[3].text.split('\n')[2].split('–')[1])
            fuel_type = bd[4].text.strip()
            transmission = bd[5].text.strip()
            engine = bd[6].text.strip()
            vin = bd[7].text.strip()
            stock_number = bd[8].text.strip()
            mileage = int(''.join(bd[9].text.split()[0].split(',')))
            
            
    
            
            
            
            new_row = pd.DataFrame([[year,make,model,used_or_new,price,consumer_rating,consumer_reviews,seller_type,seller_name,seller_rating,seller_reviews,
                                   street_name,state,zipcode,deal_type,comfort,interior_design,performance,value_for_money,exterior_styling,reliability,
                                   exterior_color,interior_color,drive_train,min_mpg,max_mpg,fuel_type,transmission,engine,vin,stock_number,mileage]],
                                  columns=columns)
            
            #Generate row in dataframe based on scraped information
            df = pd.concat([df,new_row])
        except:
            pass
    
    current_page    
    current_page.find('a', {'aria-label':'Next page'}).get('href')
    next_page = 'http://cars.com' + current_page.find('a', {'aria-label':'Next page'}).get('href') #Get link to next page
        
    url = next_page
    page = requests.get(url)
    soup = BeautifulSoup(page.text, 'lxml')
    print(i)

#file = 'C:/Users/leave/OneDrive/Documents/Scraped_Datasets/Cars_Data/cars_raw.csv'
#file = 'C:/Users/John/Desktop/BOOTCAMP/Car_Search_Project'

#df.to_csv("cars_raw.csv", index=False)
#df.to_excel("cars_raw.xlsx") 

0
1
2
3
4
5
6
7
8
9
10
11


In [6]:
df

,Year,Make,Model,Used_or_New,Price,ConsumerRating,ConsumerReviews,SellerType,SellerName,SellerRating,...,InteriorColor,Drivetrain,MinMPG,MaxMPG,FuelType,Transmission,Engine,VIN,Stock,Mileage
0,2019,BMW,X3 xDrive30i,Used,"$37,698",4.7,100,Dealer,Richfield Bloomington Honda,4.7,...,Mocha,All-wheel Drive,22,29,Gasoline,8-Speed Automatic,2.0L I4 16V GDI DOHC Turbo,5UXTR9C50KLE11030,221294A,48094
0,2019,Audi,SQ5 3.0T Premium,Used,"$55,997",4.8,7,Dealer,Audi Richfield,4.1,...,Rotor Gray,All-wheel Drive,19,24,Gasoline,8-Speed Automatic,3.0L V6 24V GDI DOHC Turbo,WA1B4AFY4K2001881,K2001881,20371
0,2020,Kia,Sportage LX,Used,"$24,998",4.8,175,Dealer,Luther Kia of Bloomington,4.8,...,Black,All-wheel Drive,22,26,Gasoline,6-Speed Automatic,2.4L I4 16V GDI DOHC,KNDPMCAC0L7651167,W9323,17094
0,2019,Audi,Q5 2.0T Premium Plus,Used,"$41,786",4.7,42,Dealer,Audi Richfield,4.1,...,Black,All-wheel Drive,22,27,Gasoline,7-Speed Automatic with Auto-Shift,2.0L I4 16V GDI DOHC Turbo,WA1BNAFY6K2052246,K2052246A,22047
0,2019,Jeep,Grand Cherokee Limited,Used,"$35,717",4.8,225,Dealer,Lupient Chevrolet,4.6,...,Black,Four-wheel Drive,18,25,Gasoline,8-Speed Automatic,3.6L V6 24V MPFI DOHC,1C4RJFBG4KC791944,LHP7987,38780
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,2017,Ford,Explorer Platinum,Used,"$34,995",4.7,364,Dealer,Juettner Ford Lincoln CJDR,4.8,...,–,Four-wheel Drive,16,22,Gasoline,6-Speed Automatic,3.5L V6 24V GDI DOHC Twin Turbo,1FM5K8HT3HGC85028,32322A,74190
0,2017,Ford,F-150 XLT,Used,"$29,904",4.8,568,Dealer,Mike Molstead Motors,4.7,...,Medium Earth Gray,4WD,15,21,Flexible Fuel,6-Speed Automatic,5.0L V8 32V MPFI DOHC Flexible Fuel,1FTFW1EF4HKE23174,5T3405C,94369
0,2019,Ford,F-150 XLT,Used,"$35,999",4.8,425,Dealer,Valu Ford Chrysler,4.8,...,Black,4WD,18,23,Gasoline,10-Speed Automatic,2.7L V6 24V PDI DOHC Twin Turbo,1FTEW1EP8KFC57780,FC57780,50139
0,2016,Ford,Escape SE,Used,"$14,999",4.6,262,Dealer,Sparta Ford,4.8,...,Charcoal Black,Front-wheel Drive,23,32,Gasoline,6-Speed Automatic,1.6L I4 16V GDI DOHC Turbo,1FMCU0GX9GUC87423,PM00017A,97305


In [7]:
df = df.drop(['ExteriorColor', 'InteriorColor'], axis=1)
df

,Year,Make,Model,Used_or_New,Price,ConsumerRating,ConsumerReviews,SellerType,SellerName,SellerRating,...,ReliabilityRating,Drivetrain,MinMPG,MaxMPG,FuelType,Transmission,Engine,VIN,Stock,Mileage
0,2019,BMW,X3 xDrive30i,Used,"$37,698",4.7,100,Dealer,Richfield Bloomington Honda,4.7,...,4.6,All-wheel Drive,22,29,Gasoline,8-Speed Automatic,2.0L I4 16V GDI DOHC Turbo,5UXTR9C50KLE11030,221294A,48094
0,2019,Audi,SQ5 3.0T Premium,Used,"$55,997",4.8,7,Dealer,Audi Richfield,4.1,...,4.9,All-wheel Drive,19,24,Gasoline,8-Speed Automatic,3.0L V6 24V GDI DOHC Turbo,WA1B4AFY4K2001881,K2001881,20371
0,2020,Kia,Sportage LX,Used,"$24,998",4.8,175,Dealer,Luther Kia of Bloomington,4.8,...,4.9,All-wheel Drive,22,26,Gasoline,6-Speed Automatic,2.4L I4 16V GDI DOHC,KNDPMCAC0L7651167,W9323,17094
0,2019,Audi,Q5 2.0T Premium Plus,Used,"$41,786",4.7,42,Dealer,Audi Richfield,4.1,...,4.7,All-wheel Drive,22,27,Gasoline,7-Speed Automatic with Auto-Shift,2.0L I4 16V GDI DOHC Turbo,WA1BNAFY6K2052246,K2052246A,22047
0,2019,Jeep,Grand Cherokee Limited,Used,"$35,717",4.8,225,Dealer,Lupient Chevrolet,4.6,...,4.8,Four-wheel Drive,18,25,Gasoline,8-Speed Automatic,3.6L V6 24V MPFI DOHC,1C4RJFBG4KC791944,LHP7987,38780
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,2017,Ford,Explorer Platinum,Used,"$34,995",4.7,364,Dealer,Juettner Ford Lincoln CJDR,4.8,...,4.7,Four-wheel Drive,16,22,Gasoline,6-Speed Automatic,3.5L V6 24V GDI DOHC Twin Turbo,1FM5K8HT3HGC85028,32322A,74190
0,2017,Ford,F-150 XLT,Used,"$29,904",4.8,568,Dealer,Mike Molstead Motors,4.7,...,4.8,4WD,15,21,Flexible Fuel,6-Speed Automatic,5.0L V8 32V MPFI DOHC Flexible Fuel,1FTFW1EF4HKE23174,5T3405C,94369
0,2019,Ford,F-150 XLT,Used,"$35,999",4.8,425,Dealer,Valu Ford Chrysler,4.8,...,4.7,4WD,18,23,Gasoline,10-Speed Automatic,2.7L V6 24V PDI DOHC Twin Turbo,1FTEW1EP8KFC57780,FC57780,50139
0,2016,Ford,Escape SE,Used,"$14,999",4.6,262,Dealer,Sparta Ford,4.8,...,4.7,Front-wheel Drive,23,32,Gasoline,6-Speed Automatic,1.6L I4 16V GDI DOHC Turbo,1FMCU0GX9GUC87423,PM00017A,97305


In [8]:
df = df.replace('Not Priced', None)
df

,Year,Make,Model,Used_or_New,Price,ConsumerRating,ConsumerReviews,SellerType,SellerName,SellerRating,...,ReliabilityRating,Drivetrain,MinMPG,MaxMPG,FuelType,Transmission,Engine,VIN,Stock,Mileage
0,2019,BMW,X3 xDrive30i,Used,"$37,698",4.7,100,Dealer,Richfield Bloomington Honda,4.7,...,4.6,All-wheel Drive,22,29,Gasoline,8-Speed Automatic,2.0L I4 16V GDI DOHC Turbo,5UXTR9C50KLE11030,221294A,48094
0,2019,Audi,SQ5 3.0T Premium,Used,"$55,997",4.8,7,Dealer,Audi Richfield,4.1,...,4.9,All-wheel Drive,19,24,Gasoline,8-Speed Automatic,3.0L V6 24V GDI DOHC Turbo,WA1B4AFY4K2001881,K2001881,20371
0,2020,Kia,Sportage LX,Used,"$24,998",4.8,175,Dealer,Luther Kia of Bloomington,4.8,...,4.9,All-wheel Drive,22,26,Gasoline,6-Speed Automatic,2.4L I4 16V GDI DOHC,KNDPMCAC0L7651167,W9323,17094
0,2019,Audi,Q5 2.0T Premium Plus,Used,"$41,786",4.7,42,Dealer,Audi Richfield,4.1,...,4.7,All-wheel Drive,22,27,Gasoline,7-Speed Automatic with Auto-Shift,2.0L I4 16V GDI DOHC Turbo,WA1BNAFY6K2052246,K2052246A,22047
0,2019,Jeep,Grand Cherokee Limited,Used,"$35,717",4.8,225,Dealer,Lupient Chevrolet,4.6,...,4.8,Four-wheel Drive,18,25,Gasoline,8-Speed Automatic,3.6L V6 24V MPFI DOHC,1C4RJFBG4KC791944,LHP7987,38780
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,2017,Ford,Explorer Platinum,Used,"$34,995",4.7,364,Dealer,Juettner Ford Lincoln CJDR,4.8,...,4.7,Four-wheel Drive,16,22,Gasoline,6-Speed Automatic,3.5L V6 24V GDI DOHC Twin Turbo,1FM5K8HT3HGC85028,32322A,74190
0,2017,Ford,F-150 XLT,Used,"$29,904",4.8,568,Dealer,Mike Molstead Motors,4.7,...,4.8,4WD,15,21,Flexible Fuel,6-Speed Automatic,5.0L V8 32V MPFI DOHC Flexible Fuel,1FTFW1EF4HKE23174,5T3405C,94369
0,2019,Ford,F-150 XLT,Used,"$35,999",4.8,425,Dealer,Valu Ford Chrysler,4.8,...,4.7,4WD,18,23,Gasoline,10-Speed Automatic,2.7L V6 24V PDI DOHC Twin Turbo,1FTEW1EP8KFC57780,FC57780,50139
0,2016,Ford,Escape SE,Used,"$14,999",4.6,262,Dealer,Sparta Ford,4.8,...,4.7,Front-wheel Drive,23,32,Gasoline,6-Speed Automatic,1.6L I4 16V GDI DOHC Turbo,1FMCU0GX9GUC87423,PM00017A,97305


In [9]:
df['Price'] = df['Price'].str.replace('$', '').str.replace(',', '')
df['Price']

<ipython-input-9-7fdb5c19a69e>:1: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will*not* be treated as literal strings when regex=True.
  df['Price'] = df['Price'].str.replace('$', '').str.replace(',', '')


0    37698
0    55997
0    24998
0    41786
0    35717
     ...  
0    34995
0    29904
0    35999
0    14999
0    27250
Name: Price, Length: 1087, dtype: object

In [10]:
df.to_csv("cars_raw.csv", index=False)
#df.to_excel("cars_raw.xlsx") 

In [13]:
describe(df)

NameError: name 'describe' is not defined